In [ ]:
#r "nuget: FSharp.Stats, 0.4.1"
#r "nuget: Plotly.NET, 2.0.0-preview.14"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.14"
#r "nuget: FSharp.Data, 4.1.1"
#r "nuget: Newtonsoft.Json, 12.0.3"
#r "nuget: QuikGraph, 2.3.0"

Installed Packages FSharp.Data, 4.1.1 FSharp.Stats, 0.4.1 Newtonsoft.Json, 12.0.3 Plotly.NET, 2.0.0-preview.14 Plotly.NET.Interactive, 2.0.0-preview.14 QuikGraph, 2.3.0

In [ ]:
#load "RecomShadowGraph.fs"

In [ ]:
open FSharp.Stats
open FSharp.Stats.Signal
open Plotly.NET.Interactive
open FSharp.Data
 
// open System.Text.Json
// open System.Text.Json.Serialization
open Newtonsoft.Json
open RecomShadowGraph

In [ ]:
let plans = CsvFile.Load(__SOURCE_DIRECTORY__ + "/6x6->3partitions.txt", hasHeaders=false).Cache()

In [ ]:
let ps = plans.Rows |> Seq.take 10 |> Seq.map (fun p -> Array2D.init 6 6 (fun i j -> p.[i*6+j]))

In [ ]:
let p = plans.Rows |> Seq.head
let P = Array2D.init 6 6 (fun i j -> p.[i*6+j])

In [ ]:
let ParsePlans = PlanParser 6

In [ ]:
ParsePlans.PlanPlot P

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let planShadows = ParsePlans.PlanShadows P

In [ ]:
planShadows |> List.map ShadowPlot

index,value
0,<!-- Plotly chart will be drawn inside this DIV -->
1,<!-- Plotly chart will be drawn inside this DIV -->
2,<!-- Plotly chart will be drawn inside this DIV -->


In [ ]:
planShadows

index,value
0,"[ 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1 ... (16 more) ]"
1,"[ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0 ... (16 more) ]"
2,"[ 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0 ... (16 more) ]"


In [ ]:
planShadows.Length

3

In [ ]:
// Fold AddNewShadows over plans.

In [ ]:
ps |> Seq.head |> printf "%A"

[["1"; "1"; "1"; "2"; "2"; "2"]
 ["1"; "3"; "1"; "2"; "1"; "2"]
 ["1"; "3"; "1"; "1"; "1"; "2"]
 ["3"; "3"; "1"; "2"; "2"; "2"]
 ["3"; "3"; "1"; "3"; "2"; "2"]
 ["3"; "3"; "3"; "3"; "3"; "2"]]

In [ ]:
let ps = plans.Rows |> Seq.map (fun p -> Array2D.init 6 6 (fun i j -> p.[i*6+j]))

In [ ]:
let shadows, i = BuildShadowNodes ps ParsePlans

In [ ]:
let s = shadows.["[[0; 0; 0; 0; 0; 0]\n [0; 0; 0; 0; 0; 0]\n [1; 1; 0; 0; 0; 0]\n [1; 1; 0; 0; 0; 0]\n [1; 0; 0; 0; 1; 0]\n [1; 1; 1; 1; 1; 1]]"]

In [ ]:
shadows.Count

106905

In [ ]:
let numPlans = shadows |> Map.map (fun k v -> v.NumberOfPlans) |> Map.toSeq |> Seq.map snd

In [ ]:
let numEdges = shadows |> Map.map (fun k v -> v.AdjacentShadowNodeIDs.Count) |> Map.toSeq |> Seq.map snd

In [ ]:
numEdges
|> fun data -> Chart.Histogram data

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
numPlans
|> fun data -> Chart.Histogram data

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
Seq.max numEdges

1559

In [ ]:
Seq.length data

106905

In [ ]:
let data = shadows |> Map.toSeq |> Seq.map snd

In [ ]:
let jsonString = JsonConvert.SerializeObject(data, Formatting.Indented)

In [ ]:
File.WriteAllText(__SOURCE_DIRECTORY__ + "/shadow_graph.json", jsonString)

In [ ]:
1 + 1

2

## Contectivity?


In [ ]:
let idIndexedMap = data |>  Seq.map (fun v -> (v.ID, v)) |> Map.ofSeq

In [ ]:
let rec DepthFirstSeach graph curNode visitedNodes = 
    match Set.contains curNode visitedNodes with 
    | true -> visitedNodes
    | false -> let visited = Set.add curNode visitedNodes;
               let adjNodes = graph |> Map.find curNode |> (fun v -> v.AdjacentShadowNodeIDs)
    
               adjNodes |> Set.remove curNode |> Set.fold (fun vis n -> DepthFirstSeach graph n vis) visited

In [ ]:
let bfs = DepthFirstSeach idIndexedMap 0 Set.empty

## Exploration / Notes / Ideas

* Are number of plans = 1 and edges = 3 identical?
  * looks like it might be?
  * 55020 in both categories.
  

### TODO
* DFS/BFS to check contiguity?

In [ ]:
let fewPlans = shadows |> Map.filter (fun k v -> v.NumberOfPlans = 1) |> Map.toSeq

In [ ]:
fewPlans |> Seq.map snd |> Seq.take 5 |> Seq.map (fun s -> ShadowPlot s.Shadow)

index,value
0,<!-- Plotly chart will be drawn inside this DIV -->
1,<!-- Plotly chart will be drawn inside this DIV -->
2,<!-- Plotly chart will be drawn inside this DIV -->
3,<!-- Plotly chart will be drawn inside this DIV -->
4,<!-- Plotly chart will be drawn inside this DIV -->


In [ ]:
let fewEdges = shadows |> Map.filter (fun k v -> v.AdjacentShadowNodeIDs.Count < 4) |> Map.toSeq

In [ ]:
fewEdges |> Seq.map snd |> Seq.take 5 |> Seq.map (fun s -> ShadowPlot s.Shadow)

index,value
0,<!-- Plotly chart will be drawn inside this DIV -->
1,<!-- Plotly chart will be drawn inside this DIV -->
2,<!-- Plotly chart will be drawn inside this DIV -->
3,<!-- Plotly chart will be drawn inside this DIV -->
4,<!-- Plotly chart will be drawn inside this DIV -->


In [ ]:
Seq.length fewEdges

55020

In [ ]:
Seq.length fewPlans

55020